In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("CNN_D0_D4.h5")
tflite_model = converter.convert()
with open('model_cifar.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path="model_cifar.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
interpreter.resize_tensor_input(input_details[0]['index'],[100, input_shape[1], input_shape[2], input_shape[3]])
interpreter.allocate_tensors()

with open('cifar_img.pkl', 'rb') as file:
    img = pickle.load(file)
with open('cifar_lab.pkl', 'rb') as file:
    lab = pickle.load(file)


input_data = np.array(img, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
acc_tf = (output_data.argmax(axis=1) == lab.reshape(-1)).astype("float").mean()
print("test accuracy: {:.4f}".format(acc_tf))